# Base Algorithm as shown in the paper

we will show the complexity calculation of each of the relevant functions and sum up the complexity for the algorithm as a whole.

In [1]:
import numpy as np
import math
import itertools
from functools import cache

In [2]:
n = 10
q = 2
x=np.random.randint(0,q,(n),dtype=np.ubyte)

**get_u_a_i:**

input - $U$ sequence metrix of size $N\times$ $n$, a char in $F_{q}$, $i$ index $\leq n$.

output - $\overline{U^{a,i}}$.

for each sequnce we check if the first occurence of $a$ is at index $i$ and if so we add the subsequence to a list without the first $i$ letters $O\left(Ni\right)$.

In [3]:
def get_u_a_i(U, a, i):
    if i == 1:
        return U[:,U[0,:] == a][1:]
    results = []
    for s in U.T:
        if len(np.where(s == a)[0]) == 0: continue
        if np.where(s == a)[0][0] == i-1:
            results.append(s)

    if len(results) == 0:
        return None
    
    return np.stack(results).T[i:]
    

# Reconstruction from subsequences algorithm:

Reconstruct a sequence $x$ of size $n$ from $N_q^-(n,t)+1$ of $x's$ subsequences of size $n-t$.

**get_composition_vector:**

returns histogram of a the input sequence $x$ $O\left(\left|x\right|\right)$.

In [4]:
def get_composition_vector(x, q):
    k = np.zeros(q,dtype=int)
    for i in x:
        k[i] += 1
    return k

**get_ordering:**

input - $N$ sized sequence, $q$ alphabet size - constant.

output - $t$ ordered permutation, $c$ ordered composition.

uses get_composition_vector $O\left(N\right)$, argsort on the histogram to return the ordering permutation $O\left(q\log\left(q\right)\right)$, and returns the ordered composition $O\left(q\right)$.

$q=2$ is a constant so get ordering has the complexity of $O\left(N\right)$.

In [5]:
def get_ordering(x, q=2):
    k = get_composition_vector(x, q)
    t = np.argsort(-k, kind='stable') # -k so it will be in descending order
    c = k[t]
    return t, c

In [6]:
k_x = get_composition_vector(x, q)
t_x, l_x = get_ordering(x, q)
print(f"x = {x}")
print(f"x's order permutation = {t_x}")
print(f"x's composition vector = {k_x}")
print(f"x's ordered composition = {l_x}")

x = [0 1 1 0 0 0 0 0 1 1]
x's order permutation = [0 1]
x's composition vector = [6 4]
x's ordered composition = [6 4]


**get_maximal_deletion_ball_size:**

input - $n$ original sequence size, $t$ deletion ball radius, $q$ alphabet size - constant.

output - maximal deletion ball size.

assuming $q=2$ the recursive call in the function will only have $O\left(1\right)$ complexity.

loops for $t+1$ iterations with counter i and in each computes ${n-t \choose i}$ since we know $t<<n$ this has the complexity $O\left(n\right)$.

in total the function has the complexity of $O\left(nt\right)$.

In [7]:
@cache
def get_maximal_deletion_ball_size(n, t, q=2):
    if n < t or t < 0:
        return 0
    if q == 1:
        return 1

    size = 0
    for i in range(t+1):
        size += math.comb(n-t, i) * get_maximal_deletion_ball_size(t, t-i, q-1)
    return size

**get_maximal_number_of_common_subsequences:**

input - $n$ original sequence size, $t$ deletion ball radius, $q$ alphabet size - constant.

output - maximal number of common subsequences.

calls get_maximal_deletion_ball_size $3$ times with $n\pm const$, $t\pm const$ which has a complexity of $O\left(3nt\right)=O\left(nt\right)$.

In [8]:
@cache
def get_maximal_number_of_common_subsequences(n, t, q=2):
    if n <= t or t <= 0:
        return 0
    return get_maximal_deletion_ball_size(n,t,q) - get_maximal_deletion_ball_size(n-1,t,q) + get_maximal_deletion_ball_size(n-2,t-1,q)

**get_subsequence_reconstruction_threshold:**

input - $n$ original sequence size, $t$ insertion ball radius, $order\_comp$ ordered composition (vec of size $q$), $q$ alphabet size - constant.

output - threshold on ordered composition using $\tau_{i}=N_{q}^{-}\left(n-i-1,\,t-i\right)$.

loops $q$ times and calls get_maximal_number_of_common_subsequences on each iteration $O\left(qnt\right)=O\left(nt\right)$.

In [9]:
def get_subsequence_reconstruction_threshold(n, t, order_comp, q=2):
    for i in range(q):
        w_i = order_comp[i]
        tau_i = get_maximal_number_of_common_subsequences(n-i-1,t-i,q)
        if w_i > tau_i:
            return i

### reconstruct_x_from_subsequences:

The following algorithm reconstructs a sequence $X \in F_q^n$ given $N_q^-(n,t)+1$ different subsequences of $x$ of size $n-t$\
The algorithm parameters:
* **n**             - The original sequence size to reconstruct.
* **subsequences**  - known subsequences of $X$, this is a ndarray of shape $(n-t, k)$ where $k$ is at least $N_q^-(n,t)+1$.
* **q**             - Size of the alphabet, the alphabets is assumed to be $[0,1,\dots, q-1]$.
* **verbose**       - control verbosity.


**Complexity:**

The algorithm loops at most $n$ times, in each iteration:

• call get_ordering, complexity $\leq O\left(N\right)$.

• call get_subsequence_reconstruction_threshold, complexity $\leq O\left(nt\right)$.

• concatenate, complexity $\leq O\left(n\right)$.

• call get_maximal_number_of_common_subsequences, complexity $\leq O\left(nt\right)$.

• call get_u_a_i (with i==1), complexity $\leq O\left(N\right)$.

getting us in total a complexity of $O\left(Nn+n^2t\right)$.

In [10]:
def reconstruct_x_from_subsequences(n, U, q=2, verbose=0):
    t = n-U.shape[0]
    reconstruction = np.array([], dtype=int)
    if verbose > 0:
        print(f"n={n}, t={t}, U.shape={U.shape}")
    iteration_count = 0

    while t >= 1:
        iteration_count += 1
        order_perm, order_comp = get_ordering(U[0], q)
        j = get_subsequence_reconstruction_threshold(n, t, order_comp, q)
        reconstruction = np.concatenate((reconstruction, order_perm[:j+1]))
        if verbose > 1:
            print(f"threshold={j}, reconstructed={reconstruction}")

        n = n-j-1
        t = t-j    
        N = get_maximal_number_of_common_subsequences(n,t,q)+1
        U = get_u_a_i(U, order_perm[j], 1)
        U = U[:,:N]
        if verbose > 1:
            print(f"n={n}, t={t}, U.shape={U.shape}")
        
    if verbose > 0:
        print(f'Took {iteration_count} iterations')

    return np.concatenate((reconstruction, U.T[0]))

Using the algorithm for q=2:

Use get_deletion_ball to construct the $t$ deletion ball of $x$, we will use $N_q^-(n,t)+1$ sequences from it to reconstruct $x$.

In [11]:
def get_deletion_ball(x, t, N=-1):
    if N == -1:
        return np.array(list(set(itertools.combinations(x, x.size-t))))
    else:
        result = set()
        for c in itertools.combinations(x, x.size-t):
            result.add(c)
            if len(result) == N:
                break

        return np.array(list(result))

In [12]:
n = 100
t = 3
q = 2

x=np.random.randint(0,q,(n),dtype=np.ubyte)
N = get_maximal_number_of_common_subsequences(n,t,q)+1
D_x = get_deletion_ball(x, t, N)

while D_x.shape[0] < N:
    x=np.random.randint(0,q,(n),dtype=np.ubyte)
    D_x = get_deletion_ball(x, t, N)

In [13]:
subsequences = get_deletion_ball(x, t, N).T

In [14]:
reconstructed_x = reconstruct_x_from_subsequences(n, subsequences, q, verbose=1)
print(f'Reconstructed {reconstructed_x}')
print(f'From {x}')
print(np.array_equal(x, reconstructed_x))

n=100, t=3, U.shape=(97, 9315)
Took 48 iterations
Reconstructed [1 0 0 0 1 0 1 1 0 0 1 1 1 1 1 0 0 1 1 1 1 0 0 0 0 1 0 0 1 0 0 1 1 1 1 0 1
 1 1 0 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 1 0 0 1 1 1 0 1 0 1 0 0 1 1 0 0 0 0
 0 0 0 0 1 1 0 0 1 1 1 0 1 1 1 0 0 1 0 1 0 1 1 0 0 1]
From [1 0 0 0 1 0 1 1 0 0 1 1 1 1 1 0 0 1 1 1 1 0 0 0 0 1 0 0 1 0 0 1 1 1 1 0 1
 1 1 0 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 1 0 0 1 1 1 0 1 0 1 0 0 1 1 0 0 0 0
 0 0 0 0 1 1 0 0 1 1 1 0 1 1 1 0 0 1 0 1 0 1 1 0 0 1]
True


In [15]:
%timeit reconstruct_x_from_subsequences(n, subsequences, q)

81.1 ms ± 2.45 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Reconstruction from supersequences algorithm:

Reconstruct a sequence $x$ of size $n$ from $N_q^+(n,t)+1$ of $x's$ supersequences of size $n+t$.

**get_maximal_insertion_ball_size:**

input - $n$ original sequence size, $t$ insertion ball radius, $q$ alphabet size - constant.

output - maximal insertion ball size.

assuming $q=2$ the recursive call in the function will only take $O\left(1\right)$ complexity.

loops for $t+1$ iterations with counter $i$ and in each computes ${n+t \choose i}$ since we know $t<<n$ this has the complexity $O\left(n\right)$.

in total the function has the complexity of $O\left(nt\right)$.

In [16]:
@cache
def get_maximal_insertion_ball_size(n, t, q=2):
    if n < 0 or t < 0:
        return 0
    size = 0
    for i in range(t+1):
        size += math.comb(n+t, i)*int(math.pow((q-1),i))
    return size

**get_maximal_number_of_common_supersequences:**

input - $n$ original sequence size, $t$ insertion ball radius.

loop $t$ times, on each loop call get_maximal_insertion_ball_size getting us an overall complexity of $O\left(nt^{2}\right)$.

In [17]:
@cache
def get_maximal_number_of_common_supersequences(n, t):
    if t <= 0:
        return 0
    result = 0
    for i in range(1, t+1):
        result += get_maximal_insertion_ball_size(n,t-i)
    return 2*result

**get_m_vector:**

input - $U$ sequence matrix of size $N\times n$, $t$ insertion ball radius, $a$ char in $F_{q}$.

output - $m\left(a\right)$ vector.

loop $t+1$ times, with counter $i$, on each loop call get_u_a_i($i+1$) with complexity of $O\left(Ni\right)$.

getting us a total complexity of $O\left(Nt^{2}\right)$.

In [18]:
def get_m_vector(U, t, a):
    m_vec = np.zeros((t+1,), dtype=np.uint)
    for i in range(t+1):
        u_a_i = get_u_a_i(U, a, i+1)
        if u_a_i is None:
            m_vec[i] = 0
        else:
            m_vec[i] = u_a_i.shape[1]

    return m_vec

**get_first_x:**

input - $U$ supersequence metrix of size $N\times n$, $t$ insertion ball radius.

output - $x_{1}$ first letter of $x$, $m\left(x_{1}\right)$ the $m$ vector of $x_{1}$ as defined in the paper. 

calls get_m_vector 2 times $O\left(Nt^{2}\right)$, sums each the two $m\left(a\right)$ vectors (which are of size $t+1$) $O\left(t\right)$, for each sequence in $U$ check if $s\in U$ belongs to $U_{0,1}$ or $U_{1,0}$ we go over all of them, and in the worst case over the entirety of each vector $O\left(Nn\right)$.

in summary we get a complexity of $O\left(Nn+Nt^{2}+t\right)=O\left(Nn+Nt^{2}\right)$.

In [19]:
def get_first_x(supersequences, t):
    N = supersequences.shape[1]
    candidates = []
    for i in range(2):
        m_v = get_m_vector(supersequences, t, i)
        if m_v.sum() == N:
            candidates.append((i, m_v))

    if len(candidates) == 1:
        return candidates[0]

    U_0_1 = 0
    U_1_0 = 0

    for s in supersequences.T:
        indices_0 = np.where(s == 0)[0]
        indices_1 = np.where(s == 1)[0]
        if len(indices_0) == 0 or len(indices_1) == 0: continue
        if indices_0[0] < indices_1[0]:
            U_0_1 += 1
        else:
            U_1_0 += 1

        # Since len(candidates)>1 then both 0 and 1 are candidates 
        # and because we added them by order candidates[0] represents 0
    if U_0_1 > U_1_0:
        return candidates[0] 
    
    return candidates[1]

**get_supersequence_reconstruction_threshold:**

input - $n$ length of sequence, $t$ insertion ball radius, $m\left(x_{1}\right)$ $m$ vector of $x_{1}$.

output - threshold for $m\left(x_{1}\right)$ with $\tau_{i}=N_{q}^{+}\left(n-1,t-i\right)$.

loop $t$ times, with counter $i$, calculate $\tau_{i}$ on each loop $O\left(ni^{2}\right)$.

getting us a total complexity $O\left(nt^{3}\right)$.

In [20]:
def get_supersequence_resonstruction_threshold(n, t, m_v):
    for i in range(t+1):
        w_i = m_v[i]
        tau_i = get_maximal_number_of_common_supersequences(n-1,t-i)
        if w_i > tau_i:
            return i

### reconstruct_x_from_supersequences:

The following algorithm reconstructs a sequence $x \in F_q^n$ given $N_q^+(n,t)+1$ different subsequences of $x$ of size $n+t$\
The algorithm parameters:
* **n**             - The original sequence size to reconstruct.
* **subsequences**  - known subsequences of $X$, this is a ndarray of shape $(n+t, k)$ where $k$ is at least $N_q^+(n,t)+1$.
* **q**             - Size of the alphabet, the alphabets is assumed to be $[0,1,\dots, q-1]$.
* **verbose**       - control verbosity.

**Complexity:**
The algorithm loops at most $n$ times:

• get_first_x, complexity $\leq O\left(Nn+Nt^{2}\right)$.

• get_supersequence_reconstruction_threshold, complexity $\leq O\left(nt^{3}\right)$.

• get_u_a_i, complexity $\leq O\left(Nt\right)$.

• get_maximal_number_of_common_supersequences, complexity $\leq O\left(nt^{2}\right)$.

• concatenations, complexity $\leq\left(n\right)$.

is summary we get a complexity of $O\left(Nn^{2}+Nnt^{2}+n^{2}t^{3}\right)$.


In [21]:
def reconstruct_x_from_supersequences(n, U, verbose=0):
    t = U.shape[0]-n
    reconstruction = np.array([], dtype=int)
    if verbose > 0:
        print(f"n={n}, t={t}, U.shape={U.shape}")
    iteration_count = 0

    while t >= 1:
        iteration_count += 1
        x_1, m_x_1 = get_first_x(U, t)
        reconstruction = np.append(reconstruction, x_1)
        if n == 1:
            U = None 
            break
        j = get_supersequence_resonstruction_threshold(n, t, m_x_1)
        if verbose > 1:
            print(f"threshold={j}, reconstructed={reconstruction}")

        U = get_u_a_i(U, x_1, j+1)
        n = n-1
        t = t-j  
        N = get_maximal_number_of_common_supersequences(n,t)+1
        U = U[:,:N]
        if verbose > 1:
            print(f"n={n}, t={t}, U.shape={U.shape}")
        
    if verbose > 0:
        print(f'Took {iteration_count} iterations')
        
    if U is None:
        return reconstruction
    return np.concatenate((reconstruction, U.T[0]))

Using the algorithm for q=2:

Use get_insertion_ball to construct the $t$ insertion ball of $x$, we will use $N_q^+(n,t)+1$ sequences from it to reconstruct $x$.

In [22]:
def get_insertion_ball(x, t, q=2, N=-1):
    result = set()
    n = x.shape[0]
    for indices in itertools.combinations_with_replacement(range(n+1), t):
        for letters in itertools.product(range(q), repeat=t):
            result.add(tuple(np.insert(x, indices, letters)))

            if len(result) == N:
                return np.array(list(result))

    return np.array(list(result))

In [23]:
n = 100
t = 3
q = 2
x=np.random.randint(0,q,(n),dtype=np.ubyte)
N = get_maximal_number_of_common_supersequences(n, t)+1
I_x = get_insertion_ball(x, t, q, N)
if I_x.shape[0] < N:
    raise Exception("Ball is too small, change parameters")


In [24]:
supersequences = get_insertion_ball(x, t, q, N).T

In [25]:
reconstructed_x = reconstruct_x_from_supersequences(n, supersequences, verbose=1)
print(f'Reconstructed {reconstructed_x}')
print(f'From {x}')
print(np.array_equal(x, reconstructed_x))

n=100, t=3, U.shape=(103, 10715)
Took 30 iterations
Reconstructed [1 0 0 0 0 1 1 1 0 0 1 0 0 1 0 0 1 0 0 1 0 1 1 0 1 1 1 0 1 1 0 0 1 1 0 1 1
 1 0 0 1 0 1 0 1 0 0 1 0 1 0 0 1 1 1 1 0 0 1 0 1 1 0 1 0 1 1 0 0 0 0 1 1 0
 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 1 1 0 0 0 1 1 0 0 1 1]
From [1 0 0 0 0 1 1 1 0 0 1 0 0 1 0 0 1 0 0 1 0 1 1 0 1 1 1 0 1 1 0 0 1 1 0 1 1
 1 0 0 1 0 1 0 1 0 0 1 0 1 0 0 1 1 1 1 0 0 1 0 1 1 0 1 0 1 1 0 0 0 0 1 1 0
 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 1 1 0 0 0 1 1 0 0 1 1]
True


In [26]:
%timeit reconstruct_x_from_supersequences(n, supersequences)

389 ms ± 11.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
